The following code was with minor modifications taken from Dubois et al. Phil Trans R Soc B 2018.

Code was made available at github:
https://github.com/adolphslab/HCP_MRI-behavior/blob/master/intelligence.ipynb


In [ ]:
from HCP_helpers import *
%load_ext rpy2.ipython
%matplotlib inline
plt.rcParams['svg.fonttype'] = 'none'

In [ ]:
%%R 
library(ggplot2)
library(psych)
library(lavaan)
library(Hmisc)
library(corrplot)
library(semPlot)
library(colorRamps)
# Helpers functions
# compute Comparative Fit Index for a factor analysis 
CFI <-function(x){
    return((1-((x$STATISTIC-x$dof))/(x$null.chisq-x$null.dof)))
}
# compute Comparative Fit Index for a bifactor analysis 
CFI_biv <-function(x){
    return((1-((x$stats$STATISTIC-x$stats$dof))/(x$stats$null.chisq-x$stats$null.dof)))
}
# compute implied matrix for a factor analysis
impliedMatrix<-function(x){
    if (dim(x$loadings)[2]==1) {
        imp      <- x$loadings %*% t(x$loadings) 
    } else {
       imp      <- x$loadings %*% x$Phi %*% t(x$loadings) 
    }
    diag(imp)<- diag(imp) + x$uniquenesses
    return(imp)
}
# compute implied matrix for a bifactor analysis
impliedMatrix_biv<-function(x){
    Gloadings     <- x$schmid$sl[,1]
    Floadings     <- x$schmid$sl[,2:(ncol(x$schmid$sl)-3)]
    uniquenesses  <- x$schmid$sl[,ncol(x$schmid$sl)-1]
    imp           <- Gloadings %*% t(Gloadings) + Floadings %*% t(Floadings)
    diag(imp)     <- diag(imp) + uniquenesses
    return(imp)
}

In [ ]:
# should figures be saved as files
exportFigs = False

In [ ]:
# config is a global variable used by several functions
# Where does the HCP data live?
config.DATADIR                 = '/path/to/data'
# Which release should subjects be selected from?
config.release                 = 'all+MEG2'
# Which resting-state denoising pipeline should be used?
# A replicates Finn et al Nature Neuroscience 2015 as closely as possible
config.pipelineName            = 'A'
# list available runs
fmriRuns                       = ['rfMRI_REST1_LR','rfMRI_REST1_RL','rfMRI_REST2_LR','rfMRI_REST2_RL']
# which file to use for the functional data?
#the code #fMRIrun# will be replaced by the appropriate run
config.fmriFileTemplate        = '#fMRIrun#_Atlas_MSMAll.dtseries.nii'
## do not alter the following lines ##
##>>>>>>>>
tmp = config.fmriFileTemplate.split('.')
if tmp[1]=='nii':
    config.isCifti = False
elif tmp[1]=='dtseries':
    config.isCifti = True
else:
    print('unknown file extension')
##<<<<<<<<
# parcellation for FC matrix
config.parcellationName        = 'Glasser' #used for easy reference
config.parcellationFile        = '/scratch/duboisjx/data/parcellations/Glasser2016/Parcels.dlabel.nii'
config.nParcels                = 360
# where are the .csv files with subject scores and info?
# unrestricted
config.behavFile               = '/path/to/HCP_data/unrestricted_....csv'
# RESTRICTED: needed for age, handedness, family structure,...
config.RbehavFile              = '/path/to/HCP_data/RESTRICTED_....csv'
# other naming conventions
config.melodicFolder           = op.join('#fMRIrun#_hp2000.ica','filtered_func_data.ica') 
config.movementRelativeRMSFile = 'Movement_RelativeRMS.txt'
config.movementRegressorsFile  = 'Movement_Regressors_dt.txt'
# it is advisable to run the analyses on a cluster with sge
config.queue        = True
parallelEnvironment = 'smp' #'openmp'
# output directory
outDir              = op.join(config.DATADIR,'Results','INTELLIGENCE',config.pipelineName,config.parcellationName)
if not op.isdir(outDir):
    makedirs(outDir)
    
# if working with volumetric data: should the parcels be restricted to the gray matter mask?
if not config.isCifti:
    config.maskParcelswithGM       = False
    if config.maskParcelswithGM:
        config.parcellationName = config.parcellationName + '_GM'

In [ ]:
# READ CSV FILES
Udf = pd.read_csv(config.behavFile)
Rdf = pd.read_csv(config.RbehavFile)
# merge unrestricted and restricted
df = pd.merge(Udf,Rdf,how='inner')
# keep only variables of interest
df = df[['Subject','Release','Gender','Age_in_Yrs','fMRI_3T_ReconVrs',
         'FS_BrainSeg_Vol','MMSE_Score',
        'Family_ID','Father_ID','Mother_ID','Race','Ethnicity','Handedness', 
        '3T_RS-fMRI_PctCompl','PMAT_Compl','NEO-FFI_Compl','MMSE_Compl',
        'Non-TB_Compl','VisProc_Compl','DelDisc_Compl','SCPT_Compl','IWRD_Compl','VSPLOT_Compl', 
        'NEOFAC_O','NEOFAC_C','NEOFAC_E','NEOFAC_A','NEOFAC_N',
        'NEORAW_01','NEORAW_02','NEORAW_03','NEORAW_04','NEORAW_05','NEORAW_06','NEORAW_07','NEORAW_08','NEORAW_09','NEORAW_10',
        'NEORAW_11','NEORAW_12','NEORAW_13','NEORAW_14','NEORAW_15','NEORAW_16','NEORAW_17','NEORAW_18','NEORAW_19','NEORAW_20',
        'NEORAW_21','NEORAW_22','NEORAW_23','NEORAW_24','NEORAW_25','NEORAW_26','NEORAW_27','NEORAW_28','NEORAW_29','NEORAW_30',
        'NEORAW_31','NEORAW_32','NEORAW_33','NEORAW_34','NEORAW_35','NEORAW_36','NEORAW_37','NEORAW_38','NEORAW_39','NEORAW_40',
        'NEORAW_41','NEORAW_42','NEORAW_43','NEORAW_44','NEORAW_45','NEORAW_46','NEORAW_47','NEORAW_48','NEORAW_49','NEORAW_50',
        'NEORAW_51','NEORAW_52','NEORAW_53','NEORAW_54','NEORAW_55','NEORAW_56','NEORAW_57','NEORAW_58','NEORAW_59','NEORAW_60',
        'CardSort_Unadj','Flanker_Unadj','ListSort_Unadj','PicSeq_Unadj','PicVocab_Unadj','ProcSpeed_Unadj','ReadEng_Unadj',
        'IWRD_TOT','PMAT24_A_CR','VSPLOT_TC'
        ]]
# replace labeled columns with dummies
df['Gender'].replace(['F','M'],[1,2],inplace=True)
df['fMRI_3T_ReconVrs'].replace(['r177','r177 r227','r227'],[1,2,3],inplace=True)

# RECOMPUTE PERSONALITY FACTOR SCORES
# NOT USED IN THIS ANALYSIS, BUT KEPT FOR CONSISTENCY WITH PERSONALITY PAPER SUBJECT SELECTION
scoring = [ 
    {'13':'n', '23':'r', '43':'n', #aesthetic interests
    '48':'r', '53':'n', '58':'n', #intellectual interests
    '03':'r', '08':'r', '18':'r', '38':'r', # unconventionality
    '28':'n', '33':'r'},#??
    {'05':'n', '10':'n', '15':'r', '30':'r', '55':'r', # orderliness
    '25':'n', '35':'n', '60':'n', # goal-striving
    '20':'n', '40':'n', '45':'r', '50':'n'}, # dependability
    {'07':'n', '12':'r', '37':'n', '42':'r', # positive affect
   '02':'n', '17':'n', '27':'r', '57':'r', # sociability
   '22':'n', '32':'n', '47':'n', '52':'n'}, # activity
    {'09':'r', '14':'r', '19':'n', '24':'r', '29':'r', '44':'r', '54':'r', '59':'r', #nonantagonistic orientation
   '04':'n', '34':'n', '39':'r', '49':'n'}, # prosocial orientation
    {'01':'r', '11':'n', '16':'r', '31':'r', '46':'r', # negative affect
   '06':'n', '21':'n', '26':'n', '36':'n', '41':'n', '51':'n', '56':'n'} # self-reproach
    ]
factors = ['O','C','E','A','N']
scoreL  = ['NEOFAC_O', 'NEOFAC_C', 'NEOFAC_E', 'NEOFAC_A_corr', 'NEOFAC_N']
diff     = list()
for iFac,factor in enumerate(factors):
    this       = np.zeros(df.shape[0])
    keyCtr = -1
    for key in scoring[iFac].keys():
        if scoring[iFac][key]=='n':
            df['NEORAW_'+key].replace(['SD','D','N','A','SA'],[0,1,2,3,4],inplace=True)
        else:
            df['NEORAW_'+key].replace(['SD','D','N','A','SA'],[4,3,2,1,0],inplace=True)
        this = this + df['NEORAW_'+key]
        keyCtr += 1
    df['NEOFAC_'+factor+'_calc']=this
    diff.append(np.sum(np.abs(df['NEOFAC_'+factor]-df['NEOFAC_'+factor+'_calc'])))
print('diffO={0:d}, diffC={1:d}, diffE={2:d}, diffA={3:d}, diffN={4:d}'.format(
    np.int(diff[0]),np.int(diff[1]),np.int(diff[2]),np.int(diff[3]),np.int(diff[4])))
# correct scores
df['NEOFAC_A_corr']  = df['NEOFAC_A_calc']

# select subjects according to release
if config.release == 'Q2':
    keepSub = (df['Release'] == 'Q2') | (df['Release'] == 'Q1')
elif config.release == 'S500':
    keepSub = (df['Release'] == 'Q3') | (df['Release'] == 'S500')
elif config.release == 'Q2+S500':
    keepSub = (df['Release'] == 'Q2') | (df['Release'] == 'Q1') | (df['Release'] == 'Q3') | (df['Release'] == 'S500')
elif config.release == 'S900':
    keepSub = (df['Release'] == 'S900')
elif config.release == 'S1200':
    keepSub = (df['Release'] == 'S1200')
elif config.release == 'all':
    keepSub = ((df['Release'] == 'Q1') | (df['Release'] == 'Q2') | (df['Release'] == 'Q3') 
           | (df['Release'] == 'S500') | (df['Release'] == 'S900') | (df['Release'] == 'S1200'))
elif config.release == 'all+MEG2':
    keepSub = ((df['Release'] == 'Q1') | (df['Release'] == 'Q2') | (df['Release'] == 'Q3') 
           | (df['Release'] == 'S500') | (df['Release'] == 'S900') | (df['Release'] == 'S1200') 
           | (df['Release'] == 'MEG2'))
else:
    sys.exit("Invalid release code")
print('Selected {} subjects for release {}'.format(np.sum(keepSub),config.release))

# select subjects that have completed all neuropsych
keepSub = keepSub & (
    (df['PMAT_Compl']==True) &
    (df['NEO-FFI_Compl']==True) &
    (df['MMSE_Compl']==True) &
    (df['Non-TB_Compl']==True) &
    (df['VisProc_Compl']==True) &
    (df['SCPT_Compl']==True) &
    (df['IWRD_Compl']==True) &
    (df['VSPLOT_Compl']==True)
    )
print('Selected {} subjects with complete neuropsych data'.format(np.sum(keepSub)))

# FURTHER EXCLUSIONARY CRITERIA: MISSING VALUES
keepSub    = np.logical_and(keepSub,np.logical_not(np.isnan(df['CardSort_Unadj'])))
keepSub    = np.logical_and(keepSub,np.logical_not(np.isnan(df['VSPLOT_TC'])))
keepSub    = np.logical_and(keepSub,np.logical_not(np.isnan(df['PicSeq_Unadj'])))
keepSub    = np.logical_and(keepSub,np.logical_not(np.isnan(df['NEORAW_01'])))
print('Kept {} subjects after removing missing values'.format(np.sum(keepSub)))

# COGNITIVE COMPROMISE --> MMSE <26 excluded
keepSub    = np.logical_and(keepSub,df['MMSE_Score']>=26)
print('Kept {} subjects after MMSE<26 exclusion criterion'.format(np.sum(keepSub)))

# PRUNE df 
df        = df[keepSub]
# reindex
df.index  = range(df.shape[0])

print('Included data for FACTOR ANALYSIS: {} subjects [{} F, {:0.1f}+/-{:0.1f} range {}-{} y.o.]'.format(
    len(df),np.sum(df['Gender']==1),np.mean(df['Age_in_Yrs']),np.std(df['Age_in_Yrs']),np.min(df['Age_in_Yrs']),np.max(df['Age_in_Yrs'])))

In [ ]:
cogScores = ['PicVocab_Unadj',              # Vocabulary, Language, Crystallized, Global
             'ReadEng_Unadj',               # Reading, Language, Crystallized, Global
             'PicSeq_Unadj',                # Episodic memory, Fluid, Global
             'Flanker_Unadj',               # Executive, Fluid, Global
             'CardSort_Unadj',              # Executive, Fluid, Global
             'ProcSpeed_Unadj',             # Speed, Executive, Fluid, Global
             'PMAT24_A_CR',                 # non-verbal reasoning: Number of Correct Responses, Median Reaction Time for Correct Responses 
             'VSPLOT_TC',                   # Spatial ability: Total Number Correct, Median Reaction Time Divided by Expected Number of Clicks for Correct 
             'IWRD_TOT',                    # Verbal memory
             'ListSort_Unadj',              # Working memory, Executive, Fluid, Global
        ]
alpha = 1e-3
for score in cogScores:
    k2, p = stats.normaltest(df[score])
    print("{} normality test: p = {:g}".format(score,p))
cogdf      = df[cogScores].copy()

# standardize scores
standardize = lambda x: (x-x.mean()) / x.std() #* 15. + 100.
cogdf = cogdf.pipe(standardize)

In [ ]:
def corrfunc(x, y, **kws):
    cmap = matplotlib.cm.get_cmap('jet')
    r, _ = stats.pearsonr(x, y)
    ax = plt.gca()
    ax.text(0.5, 0.5, "{:.2f}".format(r), size=48, ha='center', va='center')
    ax.set_axis_bgcolor(cmap((r+1)/2))
    ax.spines['left'].set_visible(False) 
    ax.spines['bottom'].set_visible(False) 
def diagfunc(x, **kws):
    ax = plt.gca()
    ylims = ax.get_ylim()
    ax.text(0.5, ylims[0]+0.9*(ylims[1]-ylims[0]), x.name, size=24, ha='center', va='center')
    ax.spines['left'].set_visible(False) 

g = sns.PairGrid(cogdf, palette=["red"])
g.map_upper(plt.scatter, s=10)
g.map_diag(sns.distplot, kde=False)
g.map_diag(diagfunc)
g.map_lower(corrfunc)

lims = (-4,4)
for ax in g.axes.flatten():
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)

if exportFigs:
    plt.savefig(op.join(outDir,"testCorr.svg"), format='svg')
    
# cmap = matplotlib.cm.get_cmap('jet')
# c = np.random.random((10,10))
# sns.heatmap(c,annot=True, fmt=".2f", cmap=cmap, vmin=-1,vmax=1)
# if exportFigs:
#     plt.savefig("/scratch/duboisjx/testCorrCBAR.svg", format='svg')

In [ ]:
%%R -i cogdf -o faValues,faSim,faSimR 

out = fa.parallel(cogdf,plot=F)#error.bars=T,se.bars=F,
faValues = out$fa.values
faSim    = out$fa.sim
faSimR   = out$fa.simr

In [ ]:
f,ax = plt.subplots(1,1,figsize=(5,5))
ax.plot(np.arange(10)+1,faSim,'r:',label='simulated data');
ax.plot(np.arange(10)+1,faSimR,'r--',label='resampled data');
ax.plot(np.arange(10)+1,faValues,'b^-',label='actual data');
ax.axhline(y=1,linestyle='-',color='k')
plt.setp(ax,xlabel='Factor #',ylabel='eigenvalues of factor analysis',xlim=(0.5,10.5));
plt.legend()

if exportFigs:
    plt.savefig(op.join(outDir,"faParallel.svg"), format='svg')

In [ ]:
%%R 
library(ggplot2)
library(psych)
library(lavaan)
library(Hmisc)
library(corrplot)
library(semPlot)
#library(colorRamps)
# Helpers functions
# compute Comparative Fit Index for a factor analysis 
CFI <-function(x){
    return((1-((x$STATISTIC-x$dof))/(x$null.chisq-x$null.dof)))
}
# compute Comparative Fit Index for a bifactor analysis 
CFI_biv <-function(x){
    return((1-((x$stats$STATISTIC-x$stats$dof))/(x$stats$null.chisq-x$stats$null.dof)))
}
# compute implied matrix for a factor analysis
impliedMatrix<-function(x){
    if (dim(x$loadings)[2]==1) {
        imp      <- x$loadings %*% t(x$loadings) 
    } else {
       imp      <- x$loadings %*% x$Phi %*% t(x$loadings) 
    }
    diag(imp)<- diag(imp) + x$uniquenesses
    return(imp)
}
# compute implied matrix for a bifactor analysis
impliedMatrix_biv<-function(x){
    Gloadings     <- x$schmid$sl[,1]
    Floadings     <- x$schmid$sl[,2:(ncol(x$schmid$sl)-3)]
    uniquenesses  <- x$schmid$sl[,ncol(x$schmid$sl)-1]
    imp           <- Gloadings %*% t(Gloadings) + Floadings %*% t(Floadings)
    diag(imp)     <- diag(imp) + uniquenesses
    return(imp)
}

In [ ]:
%%R 
fm     <- "mle"       # use maximum likelihood estimator
rotate <- "oblimin"   # use oblimin factor rotation

fitInds <- matrix(, nrow = 2, ncol = 9)
rownames(fitInds) <- c('s1','b4')
colnames(fitInds) <- c('CFI','RMSEA','SRMR','BIC','om_h','om_s1','om_s2','om_s3','om_s4')

# observed covariance matrices
obs       <-  cov(cogdf)
lobs      <-  obs[!lower.tri(obs)]

#SINGLE FACTOR
model = 1
f1     <- fa(cogdf,nfactors=1)
imp    <-  impliedMatrix(f1)
limp   <-  imp[!lower.tri(imp)]
fitInds[model,1] <-  CFI(f1)
fitInds[model,2] <-  f1$RMSEA[1]
fitInds[model,3] <-  sqrt(mean((limp - lobs)^2))
fitInds[model,4] <-  f1$BIC

# BI-FACTOR MODEL
model = 2
b4      <- omega(cogdf,nfactors=4,fm=fm,key=NULL,flip=FALSE,
        digits=3,title="Omega",sl=TRUE,labels=NULL, plot=FALSE,
        n.obs=NA,rotate=rotate,Phi = NULL,option="equal",covar=FALSE)
imp     <-  impliedMatrix_biv(b4)
limp    <-  imp[!lower.tri(imp)]
fitInds[model,1] <-  CFI_biv(b4)
fitInds[model,2] <-  b4$schmid$RMSEA[1]
fitInds[model,3] <-  sqrt(mean((limp - lobs)^2))
fitInds[model,4] <-  b4$stats$BIC
fitInds[model,5] <-  b4$omega_h
fitInds[model,6:9] <-  b4$omega.group[-1,3]

print(fitInds,digits=3)

print(b4)

In [ ]:
%%R -w 500 -h 500 -o b4Scores

diagram(b4,digits=3,cut=.2)
# export scores
b4Scores    <- factor.scores(cogdf,b4$schmid$sl[,1:5])$scores

In [ ]:
subjects = df['Subject'].values
g_factor = b4Scores[:,0]

flank=cogdf['Flanker_Unadj']
ccs = [np.corrcoef(b4Scores[:,ii],flank)[0,1] for ii in range(b4Scores.shape[1])]
which_spd = np.where(np.array(ccs) > 0.79)[0][0]
spd = b4Scores[:,which_spd]


picvoc=cogdf['PicVocab_Unadj']
ccs = [np.corrcoef(b4Scores[:,ii],picvoc)[0,1] for ii in range(b4Scores.shape[1])]
which_picvoc = np.where(np.array(ccs) > 0.83)[0][0]
cry = b4Scores[:,which_picvoc]


pmat=cogdf['PMAT24_A_CR']
ccs = [np.corrcoef(b4Scores[:,ii],pmat)[0,1] for ii in range(b4Scores.shape[1])]
which_pmat = np.where(np.array(ccs[1:]) > 0.54)[0][0]
which_pmat = which_pmat+1
vis = b4Scores[:,which_pmat]


picseq=cogdf['PicSeq_Unadj']
ccs = [np.corrcoef(b4Scores[:,ii],picseq)[0,1] for ii in range(b4Scores.shape[1])]
which_picseq = np.where(np.array(ccs) > 0.9)[0][0]
mem = b4Scores[:,which_picseq]
ccs


print(which_spd,which_picvoc,which_pmat, which_picseq)
print(ccs)

from scipy.io import savemat
savemat('g_asin_Dubois2018.mat', 
        {"g_factor" : g_factor,
         "subjects" : subjects,
         "spd" : spd,
         "cry" : cry,
         "mem" : mem,
         "vis" : vis
        })